In [ ]:
## In this example we will be changing Xemnas 1's AI so that he is beatable outside of his own arena

## It requires the following Environment Variables set 
 # USE_KH2_EDITORENGINE - Path to openkh
 # USE_KH2_GITPATH - Path to extracted KH2 ISO
 # USE_KH2_OUTPATH - Path that pnach files should be written to (should be cheats folder of PCSX2)

In [1]:
# First create the kh2lib object
from kh2lib.kh2lib import kh2lib
import os, json
lib = kh2lib()
if not os.path.isdir("workspace"):
    os.mkdir("workspace")

In [2]:
# Now we need to lookup the filename for xemnas's mdlx object (where the AI file is kept)
# This method will return information on all the different Xemnas's, so make sure to locate the correct one
lib.get_object(name="xemnas")

[{'ucm': '085C', 'mdlx': 'B_EX260', 'name': 'Armor Xemnas'},
 {'ucm': '09CA', 'mdlx': 'B_EX170_LAST_LV99', 'name': 'Data Final Xemnas'},
 {'ucm': '09C9', 'mdlx': 'B_EX170_LV99', 'name': 'Data Xemnas'},
 {'ucm': '088D', 'mdlx': 'B_EX330', 'name': 'Dragon Xemnas'},
 {'ucm': '081F', 'mdlx': 'B_EX170_LAST', 'name': 'Final Xemnas'},
 {'ucm': '0646', 'mdlx': 'B_EX170', 'name': 'Xemnas'}]

In [3]:
# Now we extract the xemnas BAR into a folder in our workspace
MDLX = "B_EX170.mdlx"
BAR = os.path.join(lib.gitpath, "KH2", "obj", MDLX)
assert os.path.isfile(BAR)
lib.editengine.bar_extract(BAR, os.path.join(os.getcwd(), "workspace","xemnas"))

In [4]:
# Open the workspace in a window and copy the b_ex.ai to have an unmodified copy to diff against
# Then open the b_ex.ai in a hex editor and make the following changes to the bottom of the file
# Find the header definition for these two functions, and replace them with 00s
#  warp_building_front
#  rc_invitation_to_dark
# Alternatively, just edit a decompiled version of this AI file in Ghidra (using Govanify's KH2 plugin)
base_file = os.path.join("workspace", "xemnas", "b_ex.ai.base")
mod_file = os.path.join("workspace", "xemnas", "b_ex.ai")

In [5]:
# Code to Roxas With Xemnas, and create the pnach file
lib.cheatengine.apply_ram_code(["11C4F008 00000646"])
lib.cheatengine.write_pnach()

In [6]:
# go to the roxas fight, and create a memdump using emuhaste
#  it is recommended to make a savestate in the cutscene before the fight loads, for ease of testing
# Place the memdump in the workspace folder, using whatever name you choose
dumpname = "xemnas_as_roxas.dump"
subdata = lib.parsingengine.load_memdump(os.path.join("workspace", dumpname))

In [7]:
# This method will look through the memdump to try and find where the ai file is located. 
# If you already know the offset where the file starts, this is uneccessary 
starting_offset = int(lib.parsingengine.locate_file(base_file),16)
hex(starting_offset)

'0xc5e97c'

In [8]:
# This code will diff the two versions of the ai file and create codes to make those same changes via RAM
codes = lib.parsingengine.generate_codes(str(starting_offset), base_file, mod_file)
# The memdump skips the first section of memory addresses so the code offsets need to be fixed
codes = [code[1].replace("0","2") + code[1:] for code in codes]
codes

['20c62f60 00000072',
 '20c62f64 00000000',
 '20c62f68 00000000',
 '20c62f6c 00000000',
 '20c62f70 00000000',
 '20c62f74 74610000',
 '20c6302c 00000000',
 '20c63030 00000000',
 '20c63034 00000000',
 '20c63038 00000000',
 '20c6303c 00000000',
 '20c63040 74620000']

In [9]:
# Now we add the codes into the pnach portion of kh2lib, and generate a pnach file in your pcsx 2 directory
# Then just load your savestate, skip cutscene and start the fight
lib.cheatengine.apply_ram_code(codes)
lib.cheatengine.write_pnach(debug=True)

patch=1,EE,11C4F008,extended,00000646

patch=1,EE,20C62F60,extended,00000072

patch=1,EE,20C62F64,extended,00000000

patch=1,EE,20C62F68,extended,00000000

patch=1,EE,20C62F6C,extended,00000000

patch=1,EE,20C62F70,extended,00000000

patch=1,EE,20C62F74,extended,74610000

patch=1,EE,20C6302C,extended,00000000

patch=1,EE,20C63030,extended,00000000

patch=1,EE,20C63034,extended,00000000

patch=1,EE,20C63038,extended,00000000

patch=1,EE,20C6303C,extended,00000000

patch=1,EE,20C63040,extended,74620000

